In [4]:
import numpy as np
from sklearn.metrics import confusion_matrix
import spacy
import os
import csv
import pandas as pd

In [ ]:
# Sample ground truth and prediction vectors
ground_truth = ['F', 'F', 'F', 'T', 'F', 'F', 'F', 'F', 'F', 'F', 'T', 'T', 'T', 'F', 'F', 'T', 'T', 'F', 'T', 'T', 'F', 'T', 'T', 'T', 'T', 'T', 'F', 'F', 'F', 'T', 'F', 'F', 'T', 'T', 'F', 'F', 'T', 'F', 'T', 'F', 'T', 'F', 'T', 'F', 'T']
prediction = ['T', 'F', 'T', 'T', 'F', 'F', 'F', 'F', 'T', 'F', 'T', 'T', 'T', 'F', 'F', 'F', 'T', 'T', 'T', 'T', 'F', 'F', 'T', 'T', 'T', 'F', 'F', 'T', 'T', 'T', 'F', 'T', 'T', 'F', 'F', 'F', 'T', 'F', 'T', 'T', 'F', 'F', 'F', 'F', 'T']

# Convert 'T' and 'F' labels to 1 and 0, respectively
ground_truth = np.array([1 if label == 'T' else 0 for label in ground_truth])
prediction = np.array([1 if label == 'T' else 0 for label in prediction])

# Calculate the confusion matrix
conf_matrix = confusion_matrix(ground_truth, prediction)

# Calculate precision and recall
tp = conf_matrix[1, 1]  # True Positives
fp = conf_matrix[0, 1]  # False Positives
fn = conf_matrix[1, 0]  # False Negatives

precision = tp / (tp + fp)
recall = tp / (tp + fn)

print("Confusion Matrix:")
print(conf_matrix)
print("Precision:", precision)
print("Recall:", recall)


# Filter Non-visual VKEs

In [5]:
specific_words = []
nonvisual_verb_list_path = "./../output/cleaned/nonvisual_verb.txt"
with open(nonvisual_verb_list_path, "r") as f:
    specific_words = f.read().splitlines()

# task = "Video-quality-classification"
# prompt_id = "1-COT-few-shot-distinguish-visual-quality-classification" 
v_list = ['Xn_7rwAHpPs', '8eBU4QfrFL4', 'rss-y2MLjWY', 'Hv0lzJZkfqU', 'XTaBAksP1d8', 'qC83Iem2KzQ', '1-jnsbRQYCM', 'V7xk-kgpZmk', 'X12nWy9hpHs', '_EHNXYrOu0o', '-EsSn8VlAUg', 'qB-E8q3yK4Q', 'ke0YxzcZ1w4', 'L7Qp33o5128', '2nyfmB9choE', 'vFmSrHcm1_0', '0Za89cR8mxg', 'dVPZF7nFafQ', 'OGilPWdaikI', 'ws-uf6IzTdk', '9FSubSIXleo', 'su7G18Flmpg', 'qEceucYcKWM', 'wzYeeEq_bpE', 'ABdevFdahy4', 'RHbDxf4EKb0', 'zfd1ZcS4sXE', 'n5OqxEroivo', 'RBmeiMZQLsA', '5N8WyIVcseM', 'fpgVDVjwUs4', 'xr7kBln3z00', '6vV7-tb6Pxc', 'L5Ehli9oqkY', 'sBeOQl4-jKQ', 'Gl5syT9o2yY', 'fEMqCktQEJo', 'TP0TzvtQSao', '1zpecSpou0Q', 'byPcWUlWiYI', 'jGuN68Z9rAY', 'GqJctGBHPSY', 'SXqqDnZDK0U', 'HiULplVM2nk', 'H9jjNPfeWmA', 'n1MIslYNju8', 'R_DwQGqzNg0', '9JTVO2jnfMM', 'A89U0coZkg4', 'McgjTFyCTfA', 'qTPKGVrFtQU', 'vFgbJomoWac', '8LX4earN0gY', 'T95ZET6snSk', 'obPT0w4LPow', 'pSlCKuSSdkA', 'nFMunlQ4ZrM', 'S1-xgIAVK4s', 'Q_EAYzJFt2g', 'TtA4Txii0us', 'GHYOyr_ISUY', 'eMiuD5ddUaA', '0fUnuFSl4jo', 'oAb2YE-5qnM', 'cdfLK8v5uXs', 'tl0fX2FnYPY', 'weTM5PlAY_Q', 'CuVmueVpJe8', 'NM08R4kWcB8', 'kMsWyPjRUZE', 'SEZp-0O7dkA', 'Avy_bnTPumw', 'CsQ2TJ8RpJE', '59hUv75jAyc', 'IsS_d1FUJRU', 'Ac533ZkHD5I', 'yxO3EONOGf8', 'mfKDaFM2Btk', 's-lSUi9LFlI', 'P5lDJYAsy_E', '4VOOVGdd6Mo', '-yGmk-7X3Fs', 'qDKU0rSnd6k', 'xbSp0TxZpM4', 'aVHdv502hR4', '0kqbw3AwgLo', 'Y6eWmpgvl2k', 'ri4oacr-ZWs', 'F93LW_P3yec', 'Y1D9slLijj0', 'TtTsNNXrRU0', '_CFPQrftMUs', 'ND9g4p7Xmvc', 'ZNSwYGkDtNY', 'xY7kVVS2HVY', 'NQU7dxNJZEs', '3epRsvmx1ws', 'Y0jNEWPsrvw', 'hUuvVrENhJk', '8SUl15KT-UE']

output_file_path = f"./../output/cleaned/1-COT-few-shot-distinguish-visual-quality-classification/"
input_file_path = "./../output/Video-quality-classification-result/1-COT-few-shot-distinguish-visual-quality-classification/"
nlp = spacy.load("en_core_web_sm")

# Extract Verbs from VKEs

In [6]:
verb_list = set()
i = 0
verb_list_path = "./../output/cleaned/verb.txt"
for vid in v_list:
    output_file_path = f"./../output/cleaned/1-COT-few-shot-distinguish-visual-quality-classification/{vid}.txt"
    input_file_path = f"./../output/Video-quality-classification-result/1-COT-few-shot-distinguish-visual-quality-classification/{vid}.txt"
    # Open the output file in write mode
    with open(output_file_path, "w") as output_file:
        with open(input_file_path, "r") as input_file:
            # Loop through each line in the file
            for line in input_file:
                words = line.split()
                if (len(words) > 3) and (words[0] == "Visual") and (words[1] == "Key") and (words[2] == "Event"):

                    nlp_line = nlp(line)
                    verbs = [token.text for token in nlp_line if token.pos_ == "VERB"]
                    if len(verbs) != 0:
                        verb_list.add(verbs[0])


In [8]:
with open(verb_list_path, "w") as verb_list_file:
    for verb in verb_list:
        verb_list_file.write(verb + "\n")

# Filter VKEs

In [ ]:
for vid in v_list:
    output_file_path = f"./../output/cleaned/1-COT-few-shot-distinguish-visual-quality-classification/{vid}.txt"
    input_file_path = f"./../output/Video-quality-classification-result/1-COT-few-shot-distinguish-visual-quality-classification/{vid}.txt"
    # Open the output file in write mode
    with open(output_file_path, "w") as output_file:
        with open(input_file_path, "r") as input_file:
            # Loop through each line in the file
            for line in input_file:
                words = line.split()
                if (len(words) > 3) and (words[0] == "Visual") and (words[1] == "Key") and (words[2] == "Event"):
                    isVisual = True
                    if words[3].endswith("(non-visual):"):
                        isVisual = False
                        continue
                    for word in specific_words:
                        if word in words:
                            isVisual = False
                            continue
                    if isVisual:
                        output_file.write(line)

In [29]:
print(specific_words)

['concludes', 'conclude', 'joke', 'described', 'instructs', 'shares', 'introduce', 'decides', 'decide', 'evaluates', 'reiterates', 'assures', 'calling', 'recalls', 'introduces', 'acknowledges', 'praised', 'repeats', 'reflect', 'recounts', 'reviews', 'summarizes', 'confirms', 'speaks', 'highlight', 'reminisces', 'announces', 'discussing', 'welcomes', 'accused', 'admires', 'discuss', 'explains', 'struggles', 'praises', 'interviews', 'asks', 'introduced', 'admired', 'plans', 'reflects', 'hoping', 'advises', 'immerses', 'refers', 'reminds', 'questions', 'mention', 'emphasizes', 'talks', 'suggests', 'express', 'mentions', 'encourages', 'discusses', 'calls', 'expresses', 'interrupts', 'describes', 'says', 'reminded', 'quotes']


# Output VKE to csv

In [10]:
# Set the path to the directory containing the text files
directory_path = "./../output/Video-quality-classification-result/1-COT-few-shot-distinguish-visual-quality-classification"

# Set the path to the output CSV file
output_file_path = "./../output/cleaned/output.csv"

# Loop through each file in the directory
with open(output_file_path, "w", newline='') as output_file:
    csv_writer = csv.writer(output_file)
    csv_writer.writerow(["vid", "VKE lsit", "GPT isVisual Label", "GPT + Keywords isVisual Label"])
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, "r") as input_file:
                # Write the filename to the first row of the CSV file
                csv_writer.writerow([filename])
                # Loop through each line in the file and write it to the CSV file
                for line in input_file:
                    words = line.split()

                    if (len(words) > 3) and (words[0] == "Visual") and (words[1] == "Key") and (words[2] == "Event"):
                        is_visual = True
                        is_GPT_visual = True

                        nlp_line = nlp(line)
                        verbs = [token.text for token in nlp_line if token.pos_ == "VERB"]

                        if(words[3].endswith("(non-visual):")):
                            is_GPT_visual = False
                            is_visual = False
                        if len(verbs) != 0:
                            if verbs[0] in specific_words:
                                is_visual = False
                        if is_visual:
                            if is_GPT_visual:
                                csv_writer.writerow(["", line.strip(), "T", "T"])
                            else:
                                csv_writer.writerow(["", line.strip(), "F", "T"])
                        else:
                            if is_GPT_visual:
                                csv_writer.writerow(["", line.strip(), "T", "F"])
                            else:
                                csv_writer.writerow(["", line.strip(), "F", "F"])


# GPT filtering confusion matrix

In [6]:
df = pd.read_csv('./../output/cleaned/output_with_ground_truth.csv')

raw_ground_truth = []
raw_gpt_prediction = []
raw_gpt_keywords_prediction = []
raw_visual_nonvisual = []
for gt, gpt_label, gpt_keywords_label, visual_nonvisual in zip(df["Ground Truth"], df["GPT isVisual Label"], df["GPT + Keywords isVisual Label"], df["Is Visual  + Non-visual"]):
    if gt in ["T", "F"]:
        raw_ground_truth.append(gt)
    if gpt_label in ["T", "F"]:
        raw_gpt_prediction.append(gpt_label)
    if gpt_keywords_label in ["T", "F"]:
        raw_gpt_keywords_prediction.append(gpt_keywords_label)
    if visual_nonvisual in ["T", "F"]:
        raw_visual_nonvisual.append(visual_nonvisual)

# Calculate the confusion matrix
gpt_keywords_matrix = confusion_matrix(raw_ground_truth, raw_gpt_keywords_prediction)
gpt_matrix = confusion_matrix(raw_ground_truth, raw_gpt_prediction)


In [7]:

# 1
print("1:")
gpt_fn = gpt_matrix[1, 0]  # actual: T, predicted: F
print("GPT False Negatives:", gpt_fn)
print("\n")

# 2
print("2:")
visual_event_num = raw_ground_truth.count("T")
visual_nonvisual_num = raw_visual_nonvisual.count("T")
precentage = visual_nonvisual_num / visual_event_num
print("Percentage of visual + non-visual:", precentage, "(", visual_nonvisual_num, "/", visual_event_num, ")")
print("\n")

# 4
print("4:")
print(gpt_keywords_matrix)
print("\n")

# 5
print("5:")
print(gpt_matrix)
print("\n")



1:
GPT False Negatives: 48


2:
Percentage of visual + non-visual: 0.2901353965183752 ( 150 / 517 )


4:
[[754  39]
 [ 61 456]]


5:
[[630 163]
 [ 48 469]]




In [11]:
tp = gpt_matrix[0, 0]  # True Positives
fp = gpt_matrix[0, 1]  # False Positives
fn = gpt_matrix[1, 0]  # False Negatives

precision = tp / (tp + fp)
recall = tp / (tp + fn)

# print("Confusion Matrix:")
# print(conf_matrix)
print("Precision:", precision)
print("Recall:", recall)

Precision: 0.7944514501891551
Recall: 0.9292035398230089


In [12]:

tp = gpt_keywords_matrix[0, 0]  # True Positives
fp = gpt_keywords_matrix[0, 1]  # False Positives
fn = gpt_keywords_matrix[1, 0]  # False Negatives

print(tp, fp, fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)

# print("Confusion Matrix:")
# print(conf_matrix)
print("Precision:", precision)
print("Recall:", recall)

754 39 61
Precision: 0.9508196721311475
Recall: 0.9251533742331288


In [43]:
len(raw_visual_nonvisual)

1310